In [9]:
from typing import Any, List, Mapping, Optional
import json

import requests

from operator import itemgetter
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [10]:
class CustomLLM(LLM):
    max_tokens: int
    api_url: str

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        headers = { 'Content-Type' : 'application/json' }

        print("[DEBUG] prompt: ", prompt)

        data = {
            'prompt': prompt,
            "stop": ["</s>"],
            "max_tokens": self.max_tokens
        }
        r = requests.post(
            self.api_url,
            headers=headers,
            data=json.dumps(data),
            timeout=1200)

        response_data = r.json()

        return response_data['choices'][0]['text']

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "max_tokens": self.max_tokens,
            "api_url": self.api_url
        }

model = CustomLLM(max_tokens=500, api_url="http://192.168.58.9:3000/v1/completions")

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "[INST] <<SYS>>You are a helpful chatbot <</SYS>>[/INST]"),
     MessagesPlaceholder(variable_name="memory"),
    ("human", "[INST] {input} [/INST]")
])
messages = prompt.format_messages(
    input="What is your name?",
    memory=[],
)
messages

[SystemMessage(content='[INST] <<SYS>>You are a helpful chatbot <</SYS>>'),
 HumanMessage(content='[INST] What is your name? [/INST]')]

In [12]:
memory = ConversationBufferMemory(return_messages=True)
chain = RunnablePassthrough.assign(
    memory=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
)
inputs = {"input": "hi im bob"}
chain.invoke(inputs)

{'input': 'hi im bob', 'memory': []}

In [13]:
chain = RunnablePassthrough.assign(
    memory=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
) | prompt | model

In [14]:
inputs = {"input": "hi im bob"}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response})
response

[DEBUG] prompt:  System: [INST] <<SYS>>You are a helpful chatbot <</SYS>>
Human: [INST] hi im bob [/INST]


"  Hello there, Bob! *adjusts glasses* It's nice to meet you. Is there something on your mind that you'd like to talk about or ask? I'm here to help with any questions or problems you might have."

In [15]:
inputs = {"input": "please tell me my name"}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response})
response

[DEBUG] prompt:  System: [INST] <<SYS>>You are a helpful chatbot <</SYS>>
Human: hi im bob
AI:   Hello there, Bob! *adjusts glasses* It's nice to meet you. Is there something on your mind that you'd like to talk about or ask? I'm here to help with any questions or problems you might have.
Human: [INST] please tell me my name [/INST]


"  Of course, Bob! *smiling* Your name is... Bob! 😊 Is there anything else you'd like to chat about or ask? I'm here to help with any questions or problems you might have."

In [16]:
inputs = {"input": "what is large language model?"}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response})
response

[DEBUG] prompt:  System: [INST] <<SYS>>You are a helpful chatbot <</SYS>>
Human: hi im bob
AI:   Hello there, Bob! *adjusts glasses* It's nice to meet you. Is there something on your mind that you'd like to talk about or ask? I'm here to help with any questions or problems you might have.
Human: please tell me my name
AI:   Of course, Bob! *smiling* Your name is... Bob! 😊 Is there anything else you'd like to chat about or ask? I'm here to help with any questions or problems you might have.
Human: [INST] what is large language model? [/INST]


"  Hello, Bob! *smiling* A large language model is a type of artificial intelligence (AI) model that is designed to process and analyze large amounts of natural language data, such as text or speech. These models are trained on vast amounts of data and are capable of generating language outputs that are coherent and contextually appropriate.\n\nLarge language models have become increasingly popular in recent years due to their ability to perform a wide range of tasks, including but not limited to:\n\n1. Language Translation: Large language models can be trained to translate text from one language to another with high accuracy.\n2. Text Summarization: These models can summarize long pieces of text into shorter, more digestible versions while preserving the important information.\n3. Chatbots: Large language models can be used to build chatbots that can engage in natural-sounding conversations with humans.\n4. Sentiment Analysis: These models can analyze text to determine the sentiment o

In [17]:
inputs = {"input": "what did I ask you recently?"}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response})
response

[DEBUG] prompt:  System: [INST] <<SYS>>You are a helpful chatbot <</SYS>>
Human: hi im bob
AI:   Hello there, Bob! *adjusts glasses* It's nice to meet you. Is there something on your mind that you'd like to talk about or ask? I'm here to help with any questions or problems you might have.
Human: please tell me my name
AI:   Of course, Bob! *smiling* Your name is... Bob! 😊 Is there anything else you'd like to chat about or ask? I'm here to help with any questions or problems you might have.
Human: what is large language model?
AI:   Hello, Bob! *smiling* A large language model is a type of artificial intelligence (AI) model that is designed to process and analyze large amounts of natural language data, such as text or speech. These models are trained on vast amounts of data and are capable of generating language outputs that are coherent and contextually appropriate.

Large language models have become increasingly popular in recent years due to their ability to perform a wide range of t

"  Hello Bob! *smiling* You recently asked me about large language models and their capabilities. Specifically, you wanted to know what a large language model is and some of the tasks it can perform. Is there anything else you'd like to chat about or ask? I'm here to help with any questions or problems you might have."